In [2]:
from crewai import Agent,Task,Crew
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] =os.getenv("GROQ_API_KEY")



## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

## Role: The Agent's Job Title 
The role specifies the agent's function or job within the crew. Think of it as a job title that outlines their area of expertise. This helps the underlying language model understand the specific persona the agent should adopt and the type of knowledge it should leverage.

**Examples of Roles:**

- "Senior Marketing Strategist"

- "Expert Travel Consultant"

- "Senior Software Engineer specializing in Python"

## Goal: The Agent's Driving Purpose 
The goal defines the agent's primary objective. It's a clear and concise statement of what the agent is meant to accomplish. This directs the agent's focus and shapes its decision-making process. A well-defined goal ensures the agent stays on track to produce the desired outcome.

**Key characteristics of an effective goal:**

- Outcome-focused: Clearly states what the agent needs to achieve.

- Includes quality standards: Sets expectations for the quality of the work.

- Defines success: Helps the agent understand what a successful outcome looks like.

**Examples of Goals:**

"Develop a comprehensive marketing plan for a new product launch."

"Create a detailed travel itinerary for a 2-week trip to Japan, focusing on cultural experiences."

"Write clean, efficient, and well-documented Python code to implement a new feature."

## Backstory: The Agent's Persona and Experience 
The backstory provides depth and context to the agent's character. It's a narrative that outlines the agent's experience, skills, and even its personality or work style. This influences how the agent approaches tasks and collaborates with other agents.

**A good backstory should:**

- Establish expertise: Detail the agent's experience and how they acquired their skills.

- Define their working style: Describe their methods and values when approaching their work.

- Create a cohesive persona: Ensure the backstory aligns with the assigned role and goal.

**Examples of Backstories:**
- "A seasoned marketing professional with 15 years of experience in launching successful tech products. You have a data-driven approach and a keen eye for market trends."

- "A passionate traveler who has explored over 50 countries. You believe in creating immersive and authentic travel experiences for your clients."

- "A meticulous software engineer with a decade of experience in building scalable and robust applications. You are a strong advocate for writing clean code and following best practices."

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
 )

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [13]:
result = crew.kickoff(inputs={"topic": "AI in healthcare"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9a79a220-5efe-4415-ad23-8d8dd03cb4af                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI in healthcare.                   │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

d:\Crew Ai\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected 
[input_value=UUID('7ddaf585-ccf7-4006-b58c-13f71ca5ba64'), input_type=UUID])
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected 
[input_value=UUID('3297ae9b-cd5d-4bfb-b9d3-7aa76e65c9cb'), input_type=UUID])
  return self.__pydantic_serializer__.to_python(

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan Document on AI in Healthcare**                                                                  │
│                                                                                                                 │
│  **1. Target Audience Analysis:**                                                                               │
│     - **Audience Type:** Healthcare professionals (doctors, nurses, healthcare administrators), patients,       │
│  technology enthusiasts, investors, and policymakers.                                                           │
│     - **Interests:**                                                                                            │
│       - Innovations in healthcare technology.                                                                   │
│       - Efficiency improvements in patient care.                                                                │
│       - Cost reduction and resource management.                                                                 │
│       - Ethical considerations of AI in healthcare.                                                             │
│     - **Pain Points:**                                                                                          │
│       - Understanding how to integrate AI solutions into existing practice.                                     │
│       - Concerns about data privacy and security.                                                               │
│       - Keeping up with rapid technological advancements.                                                       │
│       - Assessing the reliability and validity of AI-based tools.                                               │
│                                                                                                                 │
│  **2. Latest Trends, Key Players, and Noteworthy News on AI in Healthcare:**                                    │
│     - **Trends:**                                                                                               │
│       - Adoption of AI for telemedicine and remote monitoring.                                                  │
│       - Enhanced imaging diagnostics through machine learning algorithms.                                       │
│       - AI-driven personalized medicine and treatment plans.                                                    │
│       - Increased focus on health data analytics and predictive modeling.                                       │
│     - **Key Players:**                                                                                          │
│       - Companies: IBM Watson Health, Google Health, Tempus, and Philips Healthcare.                            │
│       - Institutions: Cleveland Clinic, Massachusetts Institute of Technology (MIT), and Johns Hopkins          │
│  University.                                                                                                    │
│     - **Noteworthy News:**                                                                                      │
│       - Recent studies showing AI tools accurately diagnosing diseases like cancer and diabetic retinopathy.    │
│       - FDA approvals for AI-based diagnostic tools.                                                            │
│       - Significant funding rounds for AI health startu

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7ddaf585-ccf7-4006-b58c-13f71ca5ba64                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Rising Influence of AI in Healthcare: Revolutionizing Patient Care and Beyond                            │
│                                                                                                                 │
│  Artificial intelligence (AI) stands at the forefront of technological advancements, making significant         │
│  inroads into various fields, particularly healthcare. As healthcare professionals, patients, technology        │
│  enthusiasts, and policymakers navigate the complex landscape of AI, it’s crucial to grasp its implications,    │
│  applications, and the challenges that lie ahead. This article offers an in-depth exploration of AI in          │
│  healthcare, aiming to inform stakeholders about the advancements and considerations in this rapidly evolving   │
│  field.                                                                                                         │
│                                                                                                                 │
│  AI in healthcare encompasses a range of technologies designed to replicate human intelligence and enhance      │
│  decision-making processes. Primarily, it includes machine learning, natural language processing, and robotic   │
│  process automation. These AI technologies analyze data with unprecedented speed and accuracy, enabling         │
│  healthcare providers to make informed decisions. According to the *Journal of Medical Internet Research*, AI   │
│  can significantly improve diagnostic accuracy, aiding clinicians in the early detection of diseases compared   │
│  to traditional methods.                                                                                        │
│                                                                                                                 │
│  Machine learning, a subset of AI, empowers systems to learn from data patterns without explicit programming.   │
│  This capability is crucial for efficiently processing the vast amounts of patient data generated daily.        │
│  Concurrently, natural language processing allows systems to interpret and respond to human language,           │
│  enhancing patient interaction, particularly in telemedicine settings. The seamless integration of these        │
│  technologies is redefining healthcare, making it more efficient, personalized, and accessible for all          │
│  participants.                                                                                                  │
│                                                                                                                 │
│  ## Impact of AI on Patient Care                                                                                │
│                                                                                                                 │
│  The integration of AI into patient care has already yielded promising results. Numerous hospitals that         │
│  adopted AI-powered diagnostic tools have reported enhanced accuracy in diagnosing complex conditions such as   │
│  cancers and diabetic retinopathy. A notable case study from IBM Watson Health illustrates how AI algorithms    │
│  can analyze extensive medical literature alongside pat

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d6fb5a2f-79b4-4e2f-88f3-1680bc2ea5ee                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9a79a220-5efe-4415-ad23-8d8dd03cb4af                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # The Rising Influence of AI in Healthcare: Revolutionizing Patient Care and Beyond                            │
│                                                                                                                 │
│  Artificial intelligence (AI) stands at the forefront of technological advancements, making significant         │
│  inroads into various fields, particularly healthcare. As healthcare professionals, patients, technology        │
│  enthusiasts, and policymakers navigate the complex landscape of AI, it’s crucial to grasp its implications,    │
│  applications, and the challenges that lie ahead. This article offers an in-depth exploration of AI in          │
│  healthcare, aiming to inform stakeholders about the advancements and considerations in this rapidly evolving   │
│  field.                                                                                                         │
│                                                                                                                 │
│  AI in healthcare encompasses a range of technologies designed to replicate human intelligence and enhance      │
│  decision-making processes. Primarily, it includes machine learning, natural language processing, and robotic   │
│  process automation. These AI technologies analyze data with unprecedented speed and accuracy, enabling         │
│  healthcare providers to make informed decisions. According to the *Journal of Medical Internet Research*, AI   │
│  can significantly improve diagnostic accuracy, aiding clinicians in the early detection of diseases compared   │
│  to traditional methods.                                                                                        │
│                                                                                                                 │
│  Machine learning, a subset of AI, empowers systems to learn from data patterns without explicit programming.   │
│  This capability is crucial for efficiently processing the vast amounts of patient data generated daily.        │
│  Concurrently, natural language processing allows systems to interpret and respond to human language,           │
│  enhancing patient interaction, particularly in telemedicine settings. The seamless integration of these        │
│  technologies is redefining healthcare, making it more efficient, personalized, and accessible for all          │
│  participants.                                                                                                  │
│                                                                                                                 │
│  ## Impact of AI on Patient Care                                                                                │
│                                                                                                                 │
│  The integration of AI into patient care has already yielded promising results. Numerous hospitals that         │
│  adopted AI-powered diagnostic tools have reported enhanced accuracy in diagnosing complex conditions such as   │
│  cancers and diabetic retinopathy. A notable case stud

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Rising Influence of AI in Healthcare: Revolutionizing Patient Care and Beyond

Artificial intelligence (AI) stands at the forefront of technological advancements, making significant inroads into various fields, particularly healthcare. As healthcare professionals, patients, technology enthusiasts, and policymakers navigate the complex landscape of AI, it’s crucial to grasp its implications, applications, and the challenges that lie ahead. This article offers an in-depth exploration of AI in healthcare, aiming to inform stakeholders about the advancements and considerations in this rapidly evolving field.

AI in healthcare encompasses a range of technologies designed to replicate human intelligence and enhance decision-making processes. Primarily, it includes machine learning, natural language processing, and robotic process automation. These AI technologies analyze data with unprecedented speed and accuracy, enabling healthcare providers to make informed decisions. According to the *Journal of Medical Internet Research*, AI can significantly improve diagnostic accuracy, aiding clinicians in the early detection of diseases compared to traditional methods.

Machine learning, a subset of AI, empowers systems to learn from data patterns without explicit programming. This capability is crucial for efficiently processing the vast amounts of patient data generated daily. Concurrently, natural language processing allows systems to interpret and respond to human language, enhancing patient interaction, particularly in telemedicine settings. The seamless integration of these technologies is redefining healthcare, making it more efficient, personalized, and accessible for all participants.

## Impact of AI on Patient Care

The integration of AI into patient care has already yielded promising results. Numerous hospitals that adopted AI-powered diagnostic tools have reported enhanced accuracy in diagnosing complex conditions such as cancers and diabetic retinopathy. A notable case study from IBM Watson Health illustrates how AI algorithms can analyze extensive medical literature alongside patient histories, ultimately assisting oncologists in creating tailored treatment plans for cancer patients, which has led to improved outcomes.

Moreover, remote monitoring and telemedicine—powered by AI technologies—have transformed patient care during unprecedented times. With the use of mobile applications, patients can receive real-time assistance, while healthcare providers can monitor vital signs remotely, facilitating timely interventions. The rise of these solutions has been particularly beneficial in managing chronic diseases, demonstrating that AI not only supports healthcare systems but also enhances the overall quality of life for patients.

## Challenges and Concerns

While the benefits of AI in healthcare are considerable, several challenges necessitate attention. Foremost among these are data privacy and security concerns, as the integration of AI solutions requires managing sensitive patient information. Healthcare professionals must navigate complex regulations, such as HIPAA, while ensuring that patient data remains protected from unauthorized access.

Ethical considerations associated with AI in medical decision-making present another pressing issue. Relying on AI-based systems for critical decisions raises questions about accountability and transparency in automation. Additionally, some healthcare professionals may resist adopting these technologies, fearing potential job displacement or being hesitant about depending on algorithms for patient care.

## Future Outlook of AI in Healthcare

The trajectory of AI in healthcare indicates a future rich with innovation. From predictive modeling that anticipates patient needs to AI-driven platforms that facilitate remote collaboration among healthcare teams, the possibilities are boundless. Experts predict that within the next decade, AI will enhance personalized medicine, moving beyond mere diagnosis to proactive treatment based on individual patient data.

Such advancements promise to reshape the traditional healthcare system, fostering a more interconnected and efficient ecosystem. However, for these innovations to reach their full potential, it is essential to continue discussions about ethical standards, data privacy, and the training of healthcare professionals in AI technologies.

## Conclusion

AI in healthcare presents a dual-edged sword—offering transformative benefits while also presenting significant challenges. As this technology evolves, engaging with its implications becomes essential for all stakeholders in the healthcare sector. By remaining informed about the changing landscape of AI, healthcare professionals can foster an environment that embraces innovation while upholding high ethical standards.

As the future unfolds, readers are encouraged to subscribe for updates on the latest trends in AI healthcare. This ensures they stay at the forefront of these critical developments. Furthermore, healthcare professionals should consider investing time in training and certifications related to AI technologies to enhance their expertise and seamlessly adapt to this swiftly evolving environment.
``` 

This blog post is structured to captivate the target audience, incorporate necessary SEO keywords, and provide a comprehensive examination of the topic, ensuring its readiness for publication.

: 

In [16]:
result.raw

'```markdown\n# The Rising Influence of AI in Healthcare: Revolutionizing Patient Care and Beyond\n\nArtificial intelligence (AI) stands at the forefront of technological advancements, making significant inroads into various fields, particularly healthcare. As healthcare professionals, patients, technology enthusiasts, and policymakers navigate the complex landscape of AI, it’s crucial to grasp its implications, applications, and the challenges that lie ahead. This article offers an in-depth exploration of AI in healthcare, aiming to inform stakeholders about the advancements and considerations in this rapidly evolving field.\n\nAI in healthcare encompasses a range of technologies designed to replicate human intelligence and enhance decision-making processes. Primarily, it includes machine learning, natural language processing, and robotic process automation. These AI technologies analyze data with unprecedented speed and accuracy, enabling healthcare providers to make informed decisio